<a href="https://colab.research.google.com/github/Cseudave/automatic_tops/blob/main/07_Convertir_video_a_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear los videos cortos

MoviePy es una excelente librería que permite trabajar con videos usando python. Aunque no está optimizada para utilizar tarjetas gráficas, será muy útil.  

## Instalar librerías necesarias

In [ ]:
# Instalamos librerías
!pip install google-api-python-client
!pip install pytube

!pip install moviepy
!pip install imageio-ffmpeg 
!pip install imageio
!apt install imagemagick

!pip install IPython
!pip install mpld3 

!pip install fuzzywuzzy

!pip install playsound

In [ ]:
# Actulizamos librerías
!pip install --upgrade imageio-ffmpeg
!pip install --upgrade moviepy 
!pip install --upgrade imageio
!pip install --upgrade ffmpeg

In [ ]:
# Configuración adicional para trabajar con texto en movipy usando ImageMagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
# Importamos librerías
from apiclient.discovery import build
from pytube import YouTube
import os 
import time
from bs4 import BeautifulSoup

import re
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt

import csv

from IPython.core.display import HTML
import mpld3 
from scipy.stats import moment
from moviepy.editor import *

import os 

from moviepy.audio.AudioClip import concatenate_audioclips
from moviepy.editor import concatenate_videoclips
from moviepy.editor import VideoFileClip
from moviepy.editor import ColorClip
from moviepy.editor import CompositeVideoClip
from moviepy.editor import *

from IPython.display import Audio
from datetime import datetime

## Convertir opening a un clip

In [ ]:
# Recorta el borde superior e inferior del vídeo
def recortar(clip):
  # Obtiene las dimensiones del vídeo
  clip = clip.fx(vfx.mirror_x)
  width, height = clip.size

  # Calcula las coordenadas de inicio y final del recorte
  x1 = (width - 1280) // 2
  y1 = (height - 600) // 2
  x2 = x1 + 1280
  y2 = y1 + 600

  # Recorta el vídeo
  clip = clip.crop(x1=x1, y1=y1, x2=x2, y2=y2)

  width, height = clip.size
  # clip = clip.fx(lambda frame: frame.resize((1080, 660)))
  clip = clip.fx(lambda frame: frame.resize((1080, 500)))
  width, height = clip.size
  print('Video redimensionado')
  return clip

In [ ]:
# Cortamos en tres secciones el video
# Según la información del mejor corte
def clips_x3(clip, a=30, o = 0, time=8):
  inicios = [o, a - 0.5, (a - 0.5)/2]
  finales = [i + time for i in inicios]
  print(inicios)
  print(finales)
  clips = []
  for i in range(0, 3):
    clips.append(clip.subclip(t_start=inicios[i], t_end=finales[i]))
  print('Video separado')
  return clips

In [ ]:
# Silencia dos de los tres clips
def silenciar(clips):
  clips[0] = clips[0].set_audio(None)
  clips[2] = clips[2].set_audio(None)
  return clips

In [ ]:
def montar_lienzo(clips, name):
    # Crea el lienzo de 1080 px de alto por 1920 px de ancho con fondo blanco
  canvas = ColorClip((1080, 1920), color=(14, 14, 14))

  # img = ImageClip('ops_templates/background.png')
  # img = img.set_start(0.0)
  # img = img.set_duration(clips[0].duration)


  name, shadow = text_name(name)
  # Agrega los vídeos al lienzo
  result = CompositeVideoClip([canvas.subclip(t_start=0, t_end=clips[0].duration),
                              clips[0].set_pos((0, 210)),
                              clips[1].set_pos((0, 210 + clips[0].h)),
                              clips[2].set_pos((0, 210 + 2*clips[0].h)),
                              shadow, name 
                              ])
  return result

In [ ]:
# Agrega el nombre a cada clip
def text_name(name):
  text_name = TextClip(str(name), fontsize = 125 - len(name),
                    font='MavenPro-Bold.ttf',
                    stroke_width=2, stroke_color = '#141414',
                    color = '#EEEEEE',
                    method = 'caption', size = (864, 1920),
                    kerning=-5, interline=-30, align='east') 
  text_name = text_name.set_pos((108, -540)).set_duration(6)
  text_name = text_name.crossfadein(1)
  text_name = text_name.crossfadeout(1)
  text_shadow = TextClip(str(name), fontsize = 125 - len(name), 
                  font='MavenPro-Bold.ttf',
                  stroke_width=0, stroke_color = '#111111',
                  color = '#343434',
                  method = 'caption', size = (864, 1920),
                  kerning=-5, interline=-30, align='east') 
  text_shadow = text_shadow.set_pos((111, -543)).set_duration(6)
  text_shadow = text_shadow.crossfadein(1)
  text_shadow = text_shadow.crossfadeout(1)
  
  return  text_name, text_shadow 

In [ ]:
# Crea el clip en un lienzo, agregando los tres cortes
def op_clips(nombre, a, o = 0):
  # Carga el vídeo
  filenames = os.listdir('ops_clips/')
  if str(nombre) + '_clip.mp4' in filenames:
    print('Ya tenemos el clip')
  else:
    print('Generando clip')
    clip = VideoFileClip('ops_full/' + str(nombre) + '.mp4').fx(afx.audio_normalize)
    
    clip = recortar(clip)
    clips = clips_x3(clip, a, o)
    clips = silenciar(clips) # Mejor 2
    result = montar_lienzo(clips, str(nombre))


    compression_args = {
        'codec': 'libx264',  # Codec de video a utilizar
        'bitrate': '2000k',  # Tasa de bits a utilizar
        'fps': 30,  # Cuadros por segundo
    }


    result.write_videofile('ops_clips/' + str(nombre)+'_clip.mp4',
                           **compression_args)